In [51]:
from util import *
import pandas as pd 
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)

# for LSTM (keras with tf backend)
import gzip
import os
import pickle
import requests
import time
import re
os.environ['KERAS_BACKEND']='cntk'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

In [ ]:
vizwiz_features_train_color = pd.read_csv('azure_features_images/data/vizwiz_train_color_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                    quotechar='"', error_bad_lines=False)
vizwiz_features_train_text = pd.read_csv('azure_features_images/data/vizwiz_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_features_val_color = pd.read_csv('azure_features_images/data/vizwiz_val_color_recognition.csv',
                                  delimiter=';', engine='python',
                                  dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                  quotechar='"', error_bad_lines=False)
vizwiz_features_val_text = pd.read_csv('azure_features_images/data/vizwiz_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vqa_features_train_color = pd.read_csv('azure_features_images/data/vqa_train_color_recognition.csv',
                                 delimiter=';', engine='python', 
                                 dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                 quotechar='"', error_bad_lines=False)
vqa_features_train_text = pd.read_csv('azure_features_images/data/vqa_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)
vqa_features_val_color = pd.read_csv('azure_features_images/data/vqa_val_color_recognition.csv',
                               delimiter=';', engine='python',
                               dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                               quotechar='"', error_bad_lines=False)
vqa_features_val_text = pd.read_csv('azure_features_images/data/vqa_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_targets_train = pd.read_csv('../vizwiz_skill_typ_train.csv',
                                   delimiter=',', quotechar='"',
                                   engine='python', error_bad_lines=False)
vizwiz_targets_val = pd.read_csv('../vizwiz_skill_typ_val.csv',
                                 delimiter=',', quotechar='"', engine='python', error_bad_lines=False)
vqa_targets_train = pd.read_csv('../vqa_skill_typ_train.csv',
                               engine='python', quotechar='"', error_bad_lines=False)
vqa_targets_val = pd.read_csv('../vqa_skill_typ_val.csv',
                               engine='python', quotechar='"', error_bad_lines=False)

In [3]:
vizwiz_features_train_text.head(2)

,qid,question,ocr_text,handwritten_text
0,VizWiz_train_000000000000.jpg,What's the name of this product?,"['b', 'sil', 'leaves', '0.62', 'oz', '(170)']",['NET WT O. 62 02 ( 179)']
1,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,[],['^TAKE Three 1^']


In [21]:
def join_feature_target(feature_df_text, feature_df_color, target_df):

    feature_text = copy.deepcopy(feature_df_text)
    feature_color = copy.deepcopy(feature_df_color)
    target = copy.deepcopy(target_df)

    feature_text.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_text.set_index('QID', inplace=True)

    feature_color.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_color.set_index('QID', inplace=True)

    features = feature_text.join(feature_color[['descriptions','tags','dominant_colors']],
                                 on='QID',
                                 how='outer')

    target = target[['QID', 'IMG', 'QSN', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH']]
    target.set_index('QID', inplace=True)
    target = target.astype(dtype=str)

    df = target.join(features, on='QID', how='inner')
    df['descriptions'].astype(list)

    return df

In [35]:
vizwiz_train = join_feature_target(vizwiz_features_train_text, 
                                   vizwiz_features_train_color,
                                   vizwiz_targets_train)
vizwiz_val = join_feature_target(vizwiz_features_val_text,
                                vizwiz_features_val_color,
                                vizwiz_targets_val)

In [30]:
vizwiz_train['descriptions'][0]# .split(":")[1].split(",")[0]

"[{'text': 'a bottle of wine on a table', 'confidence': 0.7408009412153109}]"

In [ ]:
vizwiz_train['descriptions'].apply(lambda x: x[0].split(":")[1].split(","))
#train_questions = vizwiz_train['QSN'].values

In [44]:
vizwiz_train.columns

Index(['IMG', 'QSN', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH', 'question', 'ocr_text',
       'handwritten_text', 'descriptions', 'tags', 'dominant_colors'],
      dtype='object')

In [53]:
#create X and Ys
features_train=vizwiz_train[['QSN', 'descriptions', 'tags', 'dominant_colors', 'handwritten_text', 'ocr_text']].values
txt_train=vizwiz_train["TXT"].values
obj_train=vizwiz_train["OBJ"].values
col_train=vizwiz_train["COL"].values
cnt_train=vizwiz_train["CNT"].values

features_train=vizwiz_train[['QSN', 'descriptions', 'tags', 'dominant_colors', 'handwritten_text', 'ocr_text']]
txt_val=vizwiz_val["TXT"].values
obj_val=vizwiz_val["OBJ"].values
col_val=vizwiz_val["COL"].values
cnt_val=vizwiz_val["CNT"].values

In [54]:
# tokenize, create seqs, pad
tok = Tokenizer(num_words=VOCAB_SIZE, lower=True, split=" ")
tok.fit_on_texts(ques_train)
train_seq = tok.texts_to_sequences(ques_train)
train_seq = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq = tok.texts_to_sequences(ques_val)
val_seq = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

In [55]:
labels = to_categorical(np.asarray(txt_train))
labels = labels.astype('float32')
print('Number of reviews by class in training set')
print(labels.sum(axis=0))
n_classes = labels.shape[1]

Number of reviews by class in training set
[6247. 8010.]


In [56]:
#Use LSTM to predict
import gzip
import numpy as np
import os
import pandas as pd
import pickle
import requests
import time

os.environ['KERAS_BACKEND']='cntk'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

In [57]:
# tokenize, create seqs, pad
tok = Tokenizer(num_words=VOCAB_SIZE, lower=True, split=" ")
tok.fit_on_texts(ques_train)
train_seq = tok.texts_to_sequences(ques_train)
train_seq = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq = tok.texts_to_sequences(ques_val)
val_seq = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

In [58]:
labels = to_categorical(np.asarray(txt_train))
labels = labels.astype('float32')
print('Number of reviews by class in training set')
print(labels.sum(axis=0))
n_classes = labels.shape[1]

Number of reviews by class in training set
[6247. 8010.]


In [59]:
import nltk 

nltk.download('punkt')

sent_lst = []

for doc in ques_train:
    sentences = nltk.tokenize.sent_tokenize(doc)
    for sent in sentences:
        word_lst = [w for w in nltk.tokenize.word_tokenize(sent) if w.isalnum()]
        sent_lst.append(word_lst)

[nltk_data] Downloading package punkt to /home/edithzeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# use skip-gram
word2vec_model = gensim.models.Word2Vec(sentences=sent_lst, min_count=6, size=EMBEDDING_DIM, sg=1, workers=os.cpu_count())

2019-02-25 22:58:43,023 : INFO : collecting all words and their counts
2019-02-25 22:58:43,024 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-25 22:58:43,035 : INFO : PROGRESS: at sentence #10000, processed 54548 words, keeping 2480 word types
2019-02-25 22:58:43,042 : INFO : collected 3155 word types from a corpus of 88885 raw words and 16214 sentences
2019-02-25 22:58:43,043 : INFO : Loading a fresh vocabulary
2019-02-25 22:58:43,045 : INFO : effective_min_count=6 retains 749 unique words (23% of original 3155, drops 2406)
2019-02-25 22:58:43,046 : INFO : effective_min_count=6 leaves 84818 word corpus (95% of original 88885, drops 4067)
2019-02-25 22:58:43,048 : INFO : deleting the raw counts dictionary of 3155 items
2019-02-25 22:58:43,049 : INFO : sample=0.001 downsamples 44 most-common words
2019-02-25 22:58:43,049 : INFO : downsampling leaves estimated 35321 word corpus (41.6% of prior 84818)
2019-02-25 22:58:43,051 : INFO : estimated required 

In [61]:
embeddings_index = {}

for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs

print('Total %s word vectors.' % len(embeddings_index))

# Initial word embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

Total 749 word vectors.
